# Implementing Support Vector Machines (SVM) from Scratch

## What are Support Vectors?

Support Vectors are the data points that lie closest to the decision boundary (hyperplane).

- They are the most critical points in the dataset because:

    - The decision boundary is defined only by these points.

    - If you remove all other points and keep only support vectors, the SVM solution stays the same.


**SVM Classifier**: `wTx + b = 0`

### Linear SVM

When data is linearly separable, SVM tries to find a straight line (2D) or hyperplane (nD) that separates classes with maximum margin.

### Non-linear SVM

In real-world data, classes are not linearly separable. Instead of computing the mapping explicitly, SVM uses the Kernel Trick.

- A kernel function computes the similarity between two points in a (possibly) higher-dimensional space without explicitly mapping them.

### Soft Margin vs Hard Margin

- **Hard Margin SVM**: No misclassifications allowed (only works with perfectly separable data).

- **Soft Margin SVM**: Allows some misclassification using slack variables 𝜉𝑖

- Controlled by C (regularization parameter):

    - High C → less tolerance for misclassification (narrow margin).

    - Low C → more tolerance (wider margin).

### For optimization we use Gradient Descent

Gradient Descent is an optimization algorithm used for minimizing the loss function in various machine learning algorithms. It is used for updating the parameters of the learning model.

- `w = w - α*dw`

- `b = b - α*db`

**Our goal is to reduce the loss function and find the best weights and bias values**

In [1]:
# Importing required libraries

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons, make_circles

In [19]:
class SupportVectorClassifier:
    """
    Support Vector Classifier for classification tasks.
    """

    # Initiating the hyperparameters
    def __init__(self, learning_rate = 0.001, lambda_param = 0.01, n_iterations = 1000):
        
        # A tuning parameter in an optimization algorithm that determines the step size at each iteration 
        self.learning_rate = learning_rate 
        # Regularization parameter to prevent overfitting
        self.lambda_param = lambda_param
        # Number of iterations for training
        self.n_iterations = n_iterations
        
    
    # Fitting the dataset to the SVM Classifier
    def fit(self, X, Y):
        
        # n_samples = number of data points, number of rows
        # n_features = number of input features, number of columns
        n_samples, n_features = X.shape

        self.w = np.zeros(n_features) # initalizing with 0
        self.b = 0 # initializing with 0

        self.X = X
        self.Y = Y

        # Implementing Gradient Descent algorithm for Optimization
        for iteration in range(self.n_iterations):
            self.update_weights()
    
    
    # Updating the weights and biases
    def update_weights(self):
        
        # In SVM, the labels are expected to be -1 and 1 
        # If it is -1 it lies on one side of the hyperplane and if it is 1 it lies on the other side of the hyperplane
        Y_label = np.where(self.Y <= 0, -1, 1)
        
        for index, xi in enumerate(self.X):
            
            # If a point is correctly classified & outside margin → update only by regularization term
            # Condition: Yi * (w*Xi - b) >= 1 | correctly classified and lies outside the margin
            if(Y_label[index] * (np.dot(xi, self.w) - self.b)) >= 1:
                dw = 2 * self.lambda_param * self.w
                db = 0

            # If a point is inside margin/misclassified → apply both regularization + hinge loss correction
            # Condition: Yi * (w*Xi - b) < 1 | Point inside margin or misclassified
            else:
                dw = 2 * self.lambda_param * self.w - (np.dot(xi, Y_label[index])) 
                db = Y_label[index]

        # Classic gradient descent update
        self.w -= self.learning_rate * dw
        self.b -= self.learning_rate * db
    

    # Predicting the label for a given input
    def predict(self, X):
        output = np.dot(X, self.w) - self.b

        # Applies the sign function, matches SVM’s labeling convention (-1 and +1)
        predicted_labels = np.sign(output)
        
        Y_hat = np.where(predicted_labels <= -1, 0, 1)

        return Y_hat